In [1]:
# IMPORTS
from ipywidgets import HBox, VBox, Label, Layout, AppLayout, Output
from IPython.display import display
import sys, os

# append parent of cwd to sys path to find modules
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# surpress pygame version prompt
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

from src.app import App
from src.datatypes.call_response_set import CallResponseSet
from dashboards.widgets import inputHeading, inputSelect, inputUpload, generationHeading, checkpointSelect, tempSlider, adaptationHeading, stepsSelect, startButton, applySettingsButton

from src.io.plot import pianoRoll
from src.io.input import loadMidiFile

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
##########################
#C#    OUTPUT & APP    ###
##########################

log = Output()
output = Output()

# init
app = App(log=log)

In [3]:
##########################
#M#   SELECTION DATA   ###
##########################

adaptation_operations = app.get_adaptation_operations()
generators = app.get_generators()
demo_melodies = app.get_demo_input()

checkpointSelect.options = [(g[0], generators.index(g)) for g in generators]
stepsSelect.options = adaptation_operations
inputSelect.options = [(d[0], demo_melodies.index(d)) for d in demo_melodies]

In [4]:
### TEST & DEBUG ###

midi = loadMidiFile('../midi/examples/monophonic/mii1_4b.mid')
midi2 = loadMidiFile('../midi/examples/monophonic/mii2_4b.mid')
midi3 = loadMidiFile('../midi/examples/monophonic/mii3_4b.mid')

In [5]:
##########################
#C#     CONTROLLER     ###
##########################

def apply_settings(b):
    app.apply_settings(checkpointSelect.value, tempSlider.value, stepsSelect.value)
    pass

def run_adaptation(b):
    # TODO handle file upload
    input_path = demo_melodies[inputSelect.value][1]
    # cr_set = app.run(input_path, False)

    # create plots from results and print them
    with output:
        output.clear_output()
        p1 = pianoRoll(midi)
        p2 = pianoRoll(midi2)
        p3 = pianoRoll(midi3)
        HBox([p1, p2, p3])


startButton.on_click(run_adaptation)
applySettingsButton.on_click(apply_settings)

In [6]:
##########################
#V#  SETTINGS SIDEBAR  ###
##########################

side_bar_layout = Layout(border='1px solid grey', padding='10px')

settings_box = VBox([
        generationHeading,
        HBox([Label('Model & Checkpoint:'), checkpointSelect]),
        HBox([Label('Temperature:'), tempSlider]),
        adaptationHeading,
        HBox([Label('Steps:'), stepsSelect]),
        applySettingsButton,
        inputHeading,
        inputSelect,
        HBox([Label('Or upload own file:'), inputUpload]),
        startButton,
    ], layout = side_bar_layout)

In [7]:
##########################
#V#    RESULTS AREA    ###
##########################

main_layout = Layout(border='1px solid grey', padding='10px')

results_box = VBox([output], layout=main_layout)

In [8]:
##########################
#V#     APP LAYOUT     ###
##########################

AppLayout(left_sidebar=settings_box,
          center=results_box,
          header=None,
          footer=log,
          right_sidebar=None,
          pane_widths=[1,2,0],
          pane_heights=[0,4,1])

AppLayout(children=(Output(layout=Layout(grid_area='footer')), VBox(children=(HTML(value='<h2>Generation Setti…